# Investigation
We will be investigating what variables will have the most impact on a house's market price.  We will be investigating from the perspective of a start up company which buys inexpensive houses and flip them for a profit.

## Objectives:

* Find what aspects of a house have a significant impact on house's value
* Discover how much a house's value will change based on a changes in condition
* Find the areas that are more profitable to sell homes. 

# Obtaining data
Time to import the necessary libraries and initialize a data frame containing the data from the CSV file.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Initializing data frame

In [2]:
df = pd.read_csv('csv_files/kc_house_data.csv')
print(df.info())
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB
None


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062


# Scrubbing the Data
Now that the data frame has been initialized, we'll start cleaning.

In [3]:
# Notes: Problems that I can immediately see are: waterfront, view, and 
# yr_renovated are missing values. Sqft_basement and dates are objects instead 
# of integer/float and datetime, respectively. 

# Checking out waterfront
df.waterfront.value_counts()
waterfront = df[(df['waterfront'] == 1.0) & (df['price'] < 300000)]
waterfront['price'].describe()


count         2.000000
mean     287500.000000
std        3535.533906
min      285000.000000
25%      286250.000000
50%      287500.000000
75%      288750.000000
max      290000.000000
Name: price, dtype: float64

In [4]:
# Looked at low end prices of waterfront properties out of curiousity. Low end
# prices too expensive to recommend to a start-up company. Drop column.
df.drop(columns='waterfront', axis=1, inplace=True)

In [5]:
# Checking out yr_renovated
df.yr_renovated.value_counts()

0.0       17011
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64

In [9]:
# Since having data about renovated homes might be useful, filling NaN's with
# zeroes
df.yr_renovated.fillna(value=0, inplace=True)

In [10]:
# view column refers to how many times the home has been viewed
print(df.view.value_counts())
df.isnull().sum()

0.0    19422
2.0      957
3.0      508
1.0      330
4.0      317
Name: view, dtype: int64


id                0
date              0
price             0
bedrooms          0
bathrooms         0
sqft_living       0
sqft_lot          0
floors            0
view             63
condition         0
grade             0
sqft_above        0
sqft_basement     0
yr_built          0
yr_renovated      0
zipcode           0
lat               0
long              0
sqft_living15     0
sqft_lot15        0
dtype: int64

In [11]:
# Only 63 missing values. Home views could potentially lead us to areas where 
# people are looking to purchase homes; advise not dropping column. This data
# only makes up 0.29% of our data. Solution: Drop rows.
df.dropna(axis=0, inplace=True)

df.isnull().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64